In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
import pickle, json
import numpy as np

import pandas as pd

In [6]:
model_type = "conversation" # abc or conversation

In [7]:
# with open(f'../data/raw/{model_type}_points.json', 'r') as f:
#     points = json.load(f)

# new_points = []
# labels = set()

# for point in points:
#     label = point['label']
#     data = point['data']

    
#     if label not in ("why"):
#         labels.add(label)
#         new_points.append({
#             'label': label,
#             'data': data
#         })

# print(labels)

# with open(f'../data/raw/{model_type}_points.json', 'w') as f:
#     json.dump(new_points, f)

In [8]:
label2id = {}
id2label = {}

def load_data():
    global NUM_LABELS

    with open(f'../data/raw/{model_type}_points.json', 'r') as f:
        points = json.load(f)

    for item in points:
        label = item["label"]
        if label not in label2id:
            label2id[label] = len(label2id)
            id2label[len(id2label)] = label

    return pd.DataFrame(points).reset_index(drop=True)

data = load_data()
data

,label,data
0,yes,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
1,yes,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
2,yes,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
3,yes,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
4,yes,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
...,...,...
2395,bad,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
2396,bad,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
2397,bad,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
2398,bad,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."


In [9]:
# Save id2label dict
with open(f"../model/{model_type}_id2label.pkl", "wb") as f:
    pickle.dump(id2label, f)

id2label

{0: 'yes',
 1: 'no',
 2: 'hello',
 3: 'bye',
 4: 'how',
 5: 'old',
 6: 'you',
 7: 'me',
 8: 'explore',
 9: 'deaf',
 10: 'good',
 11: 'bad'}

In [10]:
 # Convert labels to numbers
data['label'] = data['label'].apply(lambda x: label2id[x])
data['label'].value_counts()

0     200
1     200
2     200
3     200
4     200
5     200
6     200
7     200
8     200
9     200
10    200
11    200
Name: label, dtype: int64

In [11]:
def train_test_split(df, test_size=0.4):
    test_size = int(len(df) * test_size)
    train_size = len(df) - test_size

    # Shuffle data
    df = df.sample(frac=1).reset_index(drop=True)

    # Split data
    train = df[:train_size]
    test = df[train_size:]

    return train, test

train, test = train_test_split(data)

with open(f"../data/clean/{model_type}_train.pkl", "wb") as f:
    pickle.dump(train.to_dict('records'), f)

with open(f"../data/clean/{model_type}_test.pkl", "wb") as f:
    pickle.dump(test.to_dict('records'), f)

In [12]:
len(train), len(test)

(1440, 960)

In [13]:
train

,label,data
1278,5,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
1227,5,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
451,11,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
1326,4,"[[-0.941043496131897, 0.41338396072387695, 1.0..."
1101,6,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
...,...,...
1234,5,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
1428,11,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
1349,6,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
566,6,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."


In [14]:
test

,label,data
1524,4,"[[-1.0, 0.23193883895874023, 1.0], [-0.9833878..."
1907,9,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
1668,9,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
2108,7,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
1817,9,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
...,...,...
1654,8,"[[-1.0, 0.9694262742996216, -1.0], [-0.2177788..."
1794,2,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
1926,9,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
2013,1,"[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ..."
